# Chapter 23: Asynchronous JavaScript

---

## Introduction

Imagine you’re at a restaurant. You place your order, and then you wait—doing nothing—until your food arrives. That’s **synchronous** behavior: one task at a time, blocking everything else. Now imagine you order, then chat with friends while the kitchen prepares your meal. When the food is ready, the waiter brings it to you. That’s **asynchronous** behavior: you initiated a task and then moved on to other things, and you were notified when the task completed.

In JavaScript, many operations are asynchronous by nature:
- Fetching data from a server (AJAX)
- Reading/writing files (in Node.js)
- Timers (`setTimeout`, `setInterval`)
- User events (clicks, keypresses)

If JavaScript were purely synchronous, your entire web page would freeze every time it waited for a network response. Fortunately, JavaScript has a powerful asynchronous model built around the **event loop**, **callbacks**, **promises**, and **async/await**. Mastering these concepts is essential for creating responsive, modern web applications.

In this chapter, you will learn:

- The difference between synchronous and asynchronous code.
- How the JavaScript event loop enables concurrency.
- How to work with callbacks, promises, and async/await.
- How to make HTTP requests using the Fetch API.
- How to handle JSON data.
- Practical patterns like infinite scroll and autocomplete.

---

## 23.1 Synchronous vs Asynchronous

### Understanding Synchronous Code

In synchronous code, each statement executes one after the other, and the next statement cannot start until the previous one finishes. This is the default behavior for most JavaScript code.

```javascript
console.log('First');
console.log('Second');
console.log('Third');
// Output: First, Second, Third
```

This is straightforward, but if one operation takes a long time (e.g., a large loop or a network request), it **blocks** the entire thread. In a browser, that means the page becomes unresponsive—no scrolling, no clicking—until the operation completes.

### Why Asynchronous Code?

Asynchronous code allows the program to start a long‑running task and then continue with other work. When the task completes, the program is notified and can handle the result. This non‑blocking behavior keeps the UI responsive.

```javascript
console.log('Start');

setTimeout(() => {
    console.log('Timeout finished');
}, 2000);

console.log('End');
// Output: Start, End, (after 2 seconds) Timeout finished
```

Even though `setTimeout` takes two seconds, the `console.log('End')` runs immediately. The callback inside `setTimeout` is executed later, without blocking.

### The JavaScript Event Loop

JavaScript is single‑threaded, meaning it has only one call stack and one heap. So how does it handle asynchronous operations? The secret is the **event loop**.

**Key components:**

- **Call Stack**: Where functions are pushed when they are called, and popped when they return.
- **Web APIs**: Browser‑provided APIs (DOM events, `setTimeout`, `fetch`, etc.) that handle asynchronous tasks.
- **Callback Queue** (or Task Queue): Where callbacks from completed asynchronous tasks wait to be executed.
- **Event Loop**: Constantly checks if the call stack is empty; if it is, it takes the first callback from the callback queue and pushes it onto the call stack.

**Visual representation:**

```
┌─────────────────────────────────────────────────────────────────┐
│                         JAVASCRIPT RUNTIME                       │
│                                                                   │
│  ┌─────────────┐      ┌──────────────┐      ┌──────────────┐   │
│  │ Call Stack  │      │   Web APIs   │      │ Callback Queue│   │
│  │             │      │              │      │              │   │
│  │ function()  │ ───> │ setTimeout   │ ───> │ [callback]   │   │
│  │ function()  │      │ fetch        │      │ [callback]   │   │
│  │             │      │ DOM events   │      │              │   │
│  └─────────────┘      └──────────────┘      └──────────────┘   │
│         ▲                                                     │
│         └─────────────── Event Loop ──────────────────────────┘
│                     (checks if stack empty)                    │
└─────────────────────────────────────────────────────────────────┘
```

**Example walkthrough:**

```javascript
console.log('1');                     // (1) pushed, logs, popped

setTimeout(() => {                     // (2) pushed, setTimeout called, popped
    console.log('2');                  // (5) callback pushed to queue
}, 0);

console.log('3');                       // (3) pushed, logs, popped
// (4) call stack empty, event loop picks callback, pushes it, logs '2'
```

Even with a delay of 0, `setTimeout`'s callback is placed in the queue and only runs after the current stack is empty. This is why `'3'` logs before `'2'`.

---

## 23.2 Callbacks

### What is a Callback?

A **callback** is a function passed as an argument to another function, to be executed later when an asynchronous operation completes.

```javascript
function fetchData(callback) {
    setTimeout(() => {
        const data = { id: 1, name: 'Alice' };
        callback(data);
    }, 1000);
}

fetchData((result) => {
    console.log('Data received:', result);
});
```

### Asynchronous Callbacks

Many Web APIs use callbacks:
- `setTimeout(callback, delay)`
- `addEventListener('click', callback)`
- `fs.readFile(path, callback)` in Node.js

### Callback Hell

When you nest multiple asynchronous callbacks, the code becomes deeply indented and hard to read—a situation known as **callback hell** or the **pyramid of doom**.

```javascript
getUser(1, (user) => {
    getPosts(user.id, (posts) => {
        getComments(posts[0].id, (comments) => {
            displayComments(comments);
        });
    });
});
```

This is difficult to maintain, error‑prone, and makes error handling messy.

### Error Handling with Callbacks

A common pattern in Node.js callbacks is **error‑first** callbacks: the first argument of the callback is an error object (or `null` if no error).

```javascript
function readFile(path, callback) {
    // simulate async read
    setTimeout(() => {
        if (path === 'bad.txt') {
            callback(new Error('File not found'));
        } else {
            callback(null, 'file contents');
        }
    }, 1000);
}

readFile('good.txt', (err, data) => {
    if (err) {
        console.error('Error:', err);
    } else {
        console.log('Data:', data);
    }
});
```

Despite its usefulness, error handling in deeply nested callbacks becomes repetitive and chaotic.

---

## 23.3 Promises

A **Promise** is an object representing the eventual completion (or failure) of an asynchronous operation. Promises provide a cleaner, more robust way to handle async code compared to callbacks.

### Promise States

A promise can be in one of three states:
- **pending**: initial state, neither fulfilled nor rejected.
- **fulfilled**: the operation completed successfully.
- **rejected**: the operation failed.

Once a promise is fulfilled or rejected, it is **settled** and cannot change state.

### Creating a Promise

You create a promise using the `Promise` constructor, which takes a function (called the executor) with two parameters: `resolve` and `reject`.

```javascript
const promise = new Promise((resolve, reject) => {
    // async operation
    setTimeout(() => {
        const success = true;
        if (success) {
            resolve('Operation succeeded');
        } else {
            reject(new Error('Operation failed'));
        }
    }, 1000);
});
```

### Using Promises: `then`, `catch`, `finally`

- `then(onFulfilled, onRejected)` – attaches callbacks for fulfillment and rejection.
- `catch(onRejected)` – attaches a callback for rejection only.
- `finally(onFinally)` – executes a callback when the promise is settled (fulfilled or rejected).

```javascript
promise
    .then((result) => {
        console.log('Success:', result);
    })
    .catch((error) => {
        console.error('Error:', error);
    })
    .finally(() => {
        console.log('Promise settled');
    });
```

### Promise Chaining

One of the most powerful features of promises is **chaining**. Each `then` returns a new promise, allowing you to sequence asynchronous operations linearly.

```javascript
getUser(1)
    .then((user) => getPosts(user.id))
    .then((posts) => getComments(posts[0].id))
    .then((comments) => displayComments(comments))
    .catch((error) => console.error('Error:', error));
```

If any promise in the chain rejects, the chain skips to the nearest `catch`.

### Error Handling with Promises

You can handle errors at the end of the chain, or even inside `then` by providing a second callback. However, it's generally cleaner to use `catch`.

```javascript
getUser(1)
    .then((user) => getPosts(user.id))
    .then((posts) => {
        // handle posts, but may throw
        if (!posts.length) throw new Error('No posts');
        return getComments(posts[0].id);
    })
    .catch((error) => {
        console.error('Caught:', error);
        // You can also return a default value to continue the chain
        return [];
    })
    .then((comments) => {
        // this runs even after catch
        console.log('Comments:', comments);
    });
```

---

## 23.4 Promise Methods

The `Promise` object provides several static methods for working with multiple promises.

### `Promise.all(iterable)`

Waits for **all** promises to fulfill, or rejects immediately if any promise rejects. Returns an array of fulfillment values in the same order as the input.

```javascript
const p1 = fetch('/users/1');
const p2 = fetch('/users/2');
const p3 = fetch('/users/3');

Promise.all([p1, p2, p3])
    .then((responses) => {
        // all responses received
        return Promise.all(responses.map(r => r.json()));
    })
    .then((users) => console.log(users))
    .catch((error) => console.error('One fetch failed:', error));
```

### `Promise.allSettled(iterable)`

Waits for **all** promises to settle (fulfill or reject). Returns an array of objects describing the outcome of each promise.

```javascript
const promises = [
    fetch('/users/1'),
    fetch('/invalid-url'),
    fetch('/users/3')
];

Promise.allSettled(promises).then((results) => {
    results.forEach((result, index) => {
        if (result.status === 'fulfilled') {
            console.log(`Promise ${index} fulfilled:`, result.value);
        } else {
            console.log(`Promise ${index} rejected:`, result.reason);
        }
    });
});
```

### `Promise.race(iterable)`

Settles as soon as **any** of the promises settles (fulfills or rejects). The result (or error) is the first settled promise's outcome.

```javascript
const timeout = new Promise((_, reject) =>
    setTimeout(() => reject(new Error('Request timed out')), 5000)
);
const fetchData = fetch('/api/data');

Promise.race([fetchData, timeout])
    .then((response) => console.log('Got data'))
    .catch((error) => console.error('Error:', error));
```

### `Promise.any(iterable)` (ES2021)

Settles as soon as **any** of the promises fulfills. If all promises reject, it rejects with an `AggregateError`.

```javascript
Promise.any([
    fetch('/slow-server'),
    fetch('/fast-server'),
    fetch('/backup-server')
])
    .then((response) => console.log('First successful response:', response))
    .catch((error) => console.error('All failed:', error));
```

---

## 23.5 Async/Await

Async/await is syntactic sugar built on top of promises. It makes asynchronous code look and behave more like synchronous code, improving readability.

### `async` Functions

Declaring a function with the `async` keyword automatically makes it return a promise. Any value returned from the function is wrapped in a resolved promise; any thrown error is wrapped in a rejected promise.

```javascript
async function greet() {
    return 'Hello';
}

greet().then(console.log); // 'Hello'
```

### The `await` Keyword

`await` can only be used inside an `async` function. It pauses the execution of the function until the promise settles, and then returns the fulfilled value (or throws if rejected).

```javascript
async function getUserData(id) {
    try {
        const response = await fetch(`/users/${id}`);
        if (!response.ok) {
            throw new Error(`HTTP error! status: ${response.status}`);
        }
        const user = await response.json();
        console.log('User:', user);
        return user;
    } catch (error) {
        console.error('Failed to fetch user:', error);
        throw error; // re-throw if needed
    }
}
```

### Error Handling with `try...catch`

Using `try...catch` with async/await is much more natural than `.catch()` chains.

```javascript
async function process() {
    try {
        const user = await getUser(1);
        const posts = await getPosts(user.id);
        const comments = await getComments(posts[0].id);
        displayComments(comments);
    } catch (error) {
        console.error('Something went wrong:', error);
    }
}
```

### Parallel Async Operations

If you have multiple independent async operations, you can run them in parallel using `Promise.all` (or `Promise.allSettled`) with `await`.

```javascript
async function fetchAllUsers() {
    const userPromises = [1, 2, 3].map(id => fetch(`/users/${id}`).then(res => res.json()));
    const users = await Promise.all(userPromises);
    console.log(users);
}
```

Be careful not to accidentally serialize parallel operations:

```javascript
// ❌ Sequential - takes total time = sum of each
const user1 = await fetchUser(1);
const user2 = await fetchUser(2);
const user3 = await fetchUser(3);

// ✅ Parallel - takes total time = max of each
const [user1, user2, user3] = await Promise.all([
    fetchUser(1),
    fetchUser(2),
    fetchUser(3)
]);
```

### Top‑Level Await (ES2022)

In modules, you can use `await` outside of an `async` function. This is called **top‑level await**.

```javascript
// In a module
const response = await fetch('/api/config');
const config = await response.json();
console.log('Config loaded:', config);
```

This blocks the module's execution but not the rest of the app. It's useful for initialization.

---

## 23.6 Fetch API

The `fetch()` function is a modern, promise‑based API for making HTTP requests. It replaces the older `XMLHttpRequest`.

### Basic GET Request

```javascript
fetch('https://api.example.com/data')
    .then(response => {
        if (!response.ok) {
            throw new Error(`HTTP error! status: ${response.status}`);
        }
        return response.json(); // also .text(), .blob(), etc.
    })
    .then(data => console.log(data))
    .catch(error => console.error('Fetch error:', error));
```

### POST Request with JSON

```javascript
fetch('https://api.example.com/users', {
    method: 'POST',
    headers: {
        'Content-Type': 'application/json',
    },
    body: JSON.stringify({
        name: 'John Doe',
        email: 'john@example.com'
    })
})
    .then(response => response.json())
    .then(data => console.log('Created:', data))
    .catch(error => console.error(error));
```

### Handling Responses

The `response` object has several methods to read the body:
- `response.json()` – parses as JSON.
- `response.text()` – returns as plain text.
- `response.blob()` – for binary data (images, files).
- `response.formData()` – for `multipart/form-data`.
- `response.arrayBuffer()` – low‑level binary data.

Always check `response.ok` (true for status 200–299) or `response.status` to handle errors properly.

### Setting Headers

Headers are passed in the `headers` option as an object or `Headers` instance.

```javascript
const headers = new Headers();
headers.append('Authorization', 'Bearer my-token');
headers.append('X-Custom-Header', 'custom-value');

fetch('/api/protected', { headers })
    .then(res => res.json())
    .then(data => console.log(data));
```

### Error Handling

`fetch()` only rejects on network failure (e.g., no internet, DNS error). It **does not** reject on HTTP error status (like 404 or 500). You must check `response.ok` or `response.status` and throw yourself.

```javascript
fetch('/api/maybe-404')
    .then(response => {
        if (!response.ok) {
            throw new Error(`Request failed with ${response.status}`);
        }
        return response.json();
    })
    .catch(error => console.error('Caught:', error.message));
```

### CORS (Cross-Origin Resource Sharing)

When you request a resource from a different origin (protocol + domain + port), the browser enforces CORS. The server must include appropriate CORS headers (`Access-Control-Allow-Origin`, etc.) to allow the request. `fetch` will throw an error if CORS fails.

---

## 23.7 Working with JSON

JSON (JavaScript Object Notation) is the most common data format for web APIs.

### What is JSON?

JSON is a lightweight data‑interchange format that is easy for humans to read and write, and easy for machines to parse and generate. It looks like JavaScript objects but with stricter rules:
- Keys must be double‑quoted.
- Strings must be double‑quoted.
- No comments.
- Supports: objects, arrays, strings, numbers, booleans, null.

### `JSON.parse()`

Converts a JSON string into a JavaScript object.

```javascript
const jsonString = '{"name":"Alice","age":30,"city":"New York"}';
const obj = JSON.parse(jsonString);
console.log(obj.name); // Alice
```

### `JSON.stringify()`

Converts a JavaScript object into a JSON string.

```javascript
const obj = { name: 'Bob', age: 25, city: 'Paris' };
const jsonString = JSON.stringify(obj);
console.log(jsonString); // '{"name":"Bob","age":25,"city":"Paris"}'
```

You can also pass a replacer function or array to filter properties, and a space parameter for pretty printing.

```javascript
JSON.stringify(obj, null, 2); // pretty print with 2 spaces
```

### Handling JSON Responses with Fetch

Since `fetch` returns a stream, calling `response.json()` returns a promise that resolves with the parsed JavaScript object.

```javascript
fetch('/api/users')
    .then(response => response.json())
    .then(users => {
        // users is already a JavaScript array/object
        users.forEach(user => console.log(user.name));
    });
```

---

## 23.8 Practical AJAX Examples

Let's apply what we've learned to build real‑world patterns.

### Example 1: Loading Content Dynamically

Suppose you have a list of articles and you want to load more when a button is clicked.

```html
<div id="articles"></div>
<button id="loadMore">Load More</button>
```

```javascript
let page = 1;
const articlesDiv = document.getElementById('articles');
const loadMoreBtn = document.getElementById('loadMore');

async function loadArticles() {
    try {
        const response = await fetch(`/api/articles?page=${page}`);
        if (!response.ok) throw new Error('Failed to load');
        const articles = await response.json();

        if (articles.length === 0) {
            loadMoreBtn.disabled = true;
            loadMoreBtn.textContent = 'No more articles';
            return;
        }

        articles.forEach(article => {
            const articleEl = document.createElement('article');
            articleEl.innerHTML = `<h3>${article.title}</h3><p>${article.summary}</p>`;
            articlesDiv.appendChild(articleEl);
        });

        page++;
    } catch (error) {
        console.error('Error loading articles:', error);
    }
}

loadMoreBtn.addEventListener('click', loadArticles);
// Optionally load first page on page load
loadArticles();
```

### Example 2: Search with Autocomplete

As the user types, we fetch suggestions from the server. We'll debounce the input to avoid too many requests.

```html
<input type="text" id="search" placeholder="Search...">
<ul id="suggestions"></ul>
```

```javascript
const searchInput = document.getElementById('search');
const suggestionsList = document.getElementById('suggestions');

function debounce(func, delay) {
    let timeout;
    return function (...args) {
        clearTimeout(timeout);
        timeout = setTimeout(() => func.apply(this, args), delay);
    };
}

async function fetchSuggestions(query) {
    if (query.length < 2) {
        suggestionsList.innerHTML = '';
        return;
    }
    try {
        const response = await fetch(`/api/search?q=${encodeURIComponent(query)}`);
        if (!response.ok) throw new Error('Search failed');
        const suggestions = await response.json();
        displaySuggestions(suggestions);
    } catch (error) {
        console.error(error);
    }
}

function displaySuggestions(items) {
    suggestionsList.innerHTML = items
        .map(item => `<li>${item}</li>`)
        .join('');
    // Add click handlers etc.
}

searchInput.addEventListener('input', debounce(async (e) => {
    await fetchSuggestions(e.target.value);
}, 300));
```

### Example 3: Infinite Scroll

When the user scrolls near the bottom, automatically load more content.

```javascript
let isLoading = false;
let hasMore = true;
let currentPage = 1;

window.addEventListener('scroll', async () => {
    if (isLoading || !hasMore) return;

    const scrollY = window.scrollY;
    const windowHeight = window.innerHeight;
    const documentHeight = document.documentElement.scrollHeight;

    // If user is within 200px of bottom
    if (scrollY + windowHeight >= documentHeight - 200) {
        isLoading = true;
        try {
            const response = await fetch(`/api/items?page=${currentPage}`);
            if (!response.ok) throw new Error('Failed to load');
            const items = await response.json();

            if (items.length === 0) {
                hasMore = false;
                return;
            }

            appendItems(items);
            currentPage++;
        } catch (error) {
            console.error('Infinite scroll error:', error);
        } finally {
            isLoading = false;
        }
    }
});

function appendItems(items) {
    const container = document.getElementById('itemsContainer');
    items.forEach(item => {
        const div = document.createElement('div');
        div.className = 'item';
        div.textContent = item.name;
        container.appendChild(div);
    });
}
```

### Example 4: Form Submission with Fetch (Revisited)

Building on Chapter 22, here's a complete async form submission with loading state and error handling.

```html
<form id="contactForm">
    <input type="email" name="email" required>
    <textarea name="message" required></textarea>
    <button type="submit" id="submitBtn">Send</button>
    <div id="formStatus"></div>
</form>
```

```javascript
const form = document.getElementById('contactForm');
const submitBtn = document.getElementById('submitBtn');
const statusDiv = document.getElementById('formStatus');

form.addEventListener('submit', async (e) => {
    e.preventDefault();

    // Disable button to prevent double submission
    submitBtn.disabled = true;
    statusDiv.textContent = 'Sending...';

    try {
        const formData = new FormData(form);
        const data = Object.fromEntries(formData.entries());

        const response = await fetch('/api/contact', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify(data)
        });

        if (!response.ok) {
            throw new Error(`Server error: ${response.status}`);
        }

        const result = await response.json();
        statusDiv.textContent = 'Message sent successfully!';
        form.reset(); // optional
    } catch (error) {
        statusDiv.textContent = `Error: ${error.message}`;
    } finally {
        submitBtn.disabled = false;
    }
});
```

---

## Chapter Summary

In this chapter, you learned the fundamentals of asynchronous JavaScript, which is essential for building responsive web applications.

- **Synchronous vs Asynchronous**: Synchronous code blocks; asynchronous code allows other operations while waiting.
- **Event Loop**: The mechanism that enables concurrency in JavaScript.
- **Callbacks**: The original pattern for async code, leading to callback hell.
- **Promises**: Provide a cleaner way to handle async operations with `.then()` and `.catch()`.
- **Promise Methods**: `Promise.all`, `Promise.allSettled`, `Promise.race`, `Promise.any` for managing multiple promises.
- **Async/Await**: Syntactic sugar over promises that makes async code look synchronous.
- **Fetch API**: Modern, promise‑based way to make HTTP requests.
- **JSON**: The standard data format for web APIs.
- **Practical Examples**: Dynamic content loading, autocomplete, infinite scroll, and form submission.

### Key Takeaways

- Async code prevents the UI from freezing.
- Promises and async/await greatly improve readability and error handling over raw callbacks.
- Always handle errors in async operations—either with `.catch()` or `try/catch`.
- Use `Promise.all` for parallel independent tasks, but be mindful of error propagation.
- `fetch()` does not reject on HTTP errors; check `response.ok`.
- Debounce user input to avoid excessive network requests.

### Practice Exercises

1. Rewrite the callback‑based example from section 23.2 (getUser → getPosts → getComments) using promises, and then using async/await.

2. Create a function `fetchWithTimeout(url, timeout)` that returns a promise that rejects if the fetch takes longer than `timeout` milliseconds. Use `Promise.race`.

3. Build a simple image gallery that loads images from an API when the page scrolls near the bottom (infinite scroll). Use the Unsplash or any free image API.

4. Implement a typeahead/autocomplete input that fetches suggestions from a public API (e.g., countries, cities) and displays them. Add debouncing and keyboard navigation.

5. Use `Promise.allSettled` to fetch data from multiple endpoints (e.g., `/users`, `/posts`, `/comments`) and display a message indicating which succeeded and which failed.

---

## Coming Up Next

**Chapter 24: JavaScript and Browser APIs**

In the next chapter, we'll explore more browser APIs that JavaScript can leverage: timers, the `window` and `navigator` objects, the History API, Web Storage, and Observers (Intersection, Mutation, Resize). You'll learn how to build features like lazy loading images, detecting element visibility, and storing data locally.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='22. forms_and_javascript.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='24. javascript_and_browser_apis.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
